# Pinocchio interactive example - Robot's reachable space approximation
![](https://github.com/stack-of-tasks/pinocchio/raw/master/doc/images/pinocchio-logo-large.png)

This is an example tutorial of how to setup `pinocchio` with `pycapacity` to calculate and visualise the robot capacities

## Installing the necessary packages

Install `jupyter` to run this notebook
```
pip install jupyter
```
Install the `pinocchio` library
```
pip install pin
```
Install an additional library with robot data `example_robot_data` provided by pinocchio community as well [more info](https://github.com/Gepetto/example-robot-data)
```
pip install example-robot-data
```
Finally install the visualisation library `meshcat` that is compatible with pinocchio simple and powerful visualisaiton library [more info](https://pypi.org/project/meshcat/)
```
pip install meshcat
```
Finally install `pycapacity` for the workspace analysis
```bash
pip install pycapacity
```

For interactive visualisation
```
pip install ipywidgets
pip install ipympl
```


In [1]:
import pinocchio as pin
import numpy as np
import time

from example_robot_data import load

## visualise the robot
from pinocchio.visualize import MeshcatVisualizer

## visualise the polytope and the ellipsoid
import meshcat.geometry as g 

# import pycapacity 
import pycapacity as pycap

In [13]:
# function visualising the polutope in meshcat
def visualise_polytope(q, horizon_time =None, frame_name = None):

    if horizon_time is None:
        horizon_time = 0.2 # 200 ms by default
        
    if frame_name is None:
        frame_id = robot.model.getFrameId(robot.model.frames[-1].name)
    else:
        frame_id = robot.model.getFrameId(frame_name)

    # calculate the jacobian
    pin.framesForwardKinematics(robot.model,data,q)
    pin.computeJointJacobians(robot.model,data, q)
    J = pin.getFrameJacobian(robot.model, data, frame_id, pin.LOCAL_WORLD_ALIGNED)
    # use only position jacobian
    J = J[:3,:]
    # mass amtrix calculation
    M = pin.crba(robot.model, data, q)
    
    # end-effector pose
    Xee = data.oMf[frame_id]

    # calculate the polytope
    opt = {'calculate_faces':True}
    # calculate the polytope
    dx_poly = pycap.robot.reachable_space_approximation(
        J=J,
        M=M,
        t_min=t_min,
        t_max=t_max,
        dq_min = dq_min,
        dq_max=dq_max,
        q_min=q_min,
        q_max=q_max,
        horizon=horizon_time,
        q0 = q
    )
    # meshcat triangulated mesh
    poly = g.TriangularMeshGeometry(vertices=dx_poly.vertices.T + Xee.translation, faces=dx_poly.face_indices)
    viz.viewer['poly'].set_object(poly, g.MeshBasicMaterial(color=0x0022ff, wireframe=True, linewidth=3, opacity=0.2))

In [3]:

# get panda robot usinf example_robot_data
# try out other robots using: load('ur5') # load('ur10')
robot = load('panda') 
data = robot.model.createData()

# get joint position ranges
q_max = robot.model.upperPositionLimit.T
q_min = robot.model.lowerPositionLimit.T
# get max velocity
dq_max = robot.model.velocityLimit
dq_min = -dq_max
# get max torque
t_max = robot.model.effortLimit.T
t_min = -t_max

# initial horizon time
horizon_time = 0.2 #seconds

# Use robot configuration.
# q0 = np.random.uniform(q_min,q_max)
q = (q_min+q_max)/2

viz = MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
# Start a new MeshCat server and client.
viz.initViewer(open=True)
# Load the robot in the viewer.
viz.loadViewerModel()
viz.display(q)
# small time window for loading the model 
# if meshcat does not visualise the robot properly, augment the time
# it can be removed in most cases
time.sleep(0.2) 

viz.viewer.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [6]:
from ipywidgets import interact, FloatSlider
kwargs = {'q[{}]'.format(i) : 
          FloatSlider(
              min = q_min[i], 
              max = q_max[i], 
              step = 0.01, 
              value = q[i]) 
          for i,q_1 in enumerate(q)}
kwargs = kwargs | {'horizon time' :  FloatSlider( min = 0.01, max = 1, step = 0.01, value = horizon_time)}
@interact(**kwargs)
def update(**kwargs):
    q = np.array([v  for v in kwargs.values()])
    viz.display(q[:-1])
    horizon_time = q[-1]
    visualise_polytope(q[:-1], horizon_time)


interactive(children=(FloatSlider(value=0.0, description='q[0]', max=2.8973, min=-2.8973, step=0.01), FloatSli…